In [1]:
from imutils.perspective import four_point_transform
from imutils import contours
import imutils
import cv2
import numpy as np
import argpa rse
import random
import matplotlib.pyplot as plt
# !pip install imutils

In [2]:
def show_images(images, titles, kill_later=True):
    for index, image in enumerate(images):
        cv2.imshow(titles[index], image)
    cv2.waitKey(0)
    if kill_later:
        cv2.destroyAllWindows()

In [3]:
# edge detection
image = cv2.imread( "images/scan_test.jpg")
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
blurred = cv2.GaussianBlur(gray, (5, 5), 0)
edged = cv2.Canny(blurred, 75, 200)
show_images([edged], ["Edged"])

In [4]:
# find contours in edge detected image
cnts = cv2.findContours(edged, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts = imutils.grab_contours(cnts)
docCnt = None

allContourImage = image.copy()
cv2.drawContours(allContourImage, cnts, -1, (0, 0, 255), 3)
print("Total contours found after edge detection {}".format(len(cnts)))
show_images([allContourImage], ["All contours from edge detected image"])

Total contours found after edge detection 18


In [5]:
# finding the document contour
if len(cnts) > 0:
    cnts = sorted(cnts, key=cv2.contourArea, reverse=True)

    for c in cnts:
        peri = cv2.arcLength(c, closed=True)
        approx = cv2.approxPolyDP(c, epsilon=peri*0.02, closed=True)

        if len(approx) == 4:
            docCnt = approx
            break

contourImage = image.copy()
cv2.drawContours(contourImage, [docCnt], -1, (0, 0, 255), 2)
show_images([contourImage], ["Outline"])

In [6]:
# Getting the bird's eye view, top-view of the document
paper = four_point_transform(image, docCnt.reshape(4, 2))
warped = four_point_transform(gray, docCnt.reshape(4, 2))
show_images([paper, warped], ["Paper", "Gray"])

In [7]:
# Thresholding the document
thresh = cv2.threshold(warped, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
show_images([thresh], ["Thresh"])

In [9]:
# Finding contours in threshold image
cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts = imutils.grab_contours(cnts)
print("Total contours found after threshold {}".format(len(cnts)))
questionCnts = []
allContourImage = paper.copy()
cv2.drawContours(allContourImage, cnts, -1, (0, 0, 255), 3)
show_images([allContourImage], ["All contours from threshold image"])

Total contours found after threshold 1


In [10]:
# Finding the questions contours
for c in cnts:
    (x, y, w, h) = cv2.boundingRect(c)
    ar = w / float(h)

    if w >= 20 and h >= 20 and ar >= 0.9 and ar <= 1.1:
        questionCnts.append(c)

print("Total questions contours found: {}".format(len(questionCnts)))

questionsContourImage = paper.copy()
cv2.drawContours(questionsContourImage, questionCnts, -1, (0, 0, 255), 3)
show_images([questionsContourImage], ["All questions contours after filtering questions"])

Total questions contours found: 0


In [ ]:
# Sorting the contours according to the question
questionCnts = contours.sort_contours(questionCnts, method="top-to-bottom")[0]
correct = 0
questionsContourImage = paper.copy()

for (q, i) in enumerate(np.arange(0, len(questionCnts), 5)):
    cnts = contours.sort_contours(questionCnts[i: i+5])[0]
    cv2.drawContours(questionsContourImage, cnts, -1, (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)), 2)
    bubbled = None

    for (j, c) in enumerate(cnts):
        mask = np.zeros(thresh.shape, dtype="uint8")
        cv2.drawContours(mask, [c], -1, 255, -1)

        mask = cv2.bitwise_and(thresh, thresh, mask=mask)
        show_images([mask], ["Mask of question {} for row {}".format(j+1, q+1)])
        total = cv2.countNonZero(mask)

        if bubbled is None or total > bubbled[0]:
            bubbled = (total, j)

    color = (0, 0, 255)
    k = ANSWER_KEY[q]

    if k == bubbled[1]:
        color = (0, 255, 0)
        correct += 1

    cv2.drawContours(paper, [cnts[k]], -1, color, 3)

show_images([questionsContourImage], ["All questions contours with different colors"])

In [ ]:
score = (correct / 5.0) * 100
print("INFO Score: {:.2f}%".format(score))
cv2.putText(paper, "{:.2f}%".format(score), (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)
show_images([image, paper], ["Original", "exam"])

In [ ]:
# def grader(image, ANSWER_KEY):

#     # edge detection
#     gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
#     blurred = cv2.GaussianBlur(gray, (5, 5), 0)
#     edged = cv2.Canny(blurred, 75, 200)
#     # show_images([edged], ["Edged"])

#     # find contours in edge detected image
#     cnts = cv2.findContours(edged, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
#     cnts = imutils.grab_contours(cnts)
#     docCnt = None

#     allContourImage = image.copy()
#     cv2.drawContours(allContourImage, cnts, -1, (0, 0, 255), 3)
#     print("Total contours found after edge detection {}".format(len(cnts)))
#     # show_images([allContourImage], ["All contours from edge detected image"])

#     # finding the document contour
#     if len(cnts) > 0:
#         cnts = sorted(cnts, key=cv2.contourArea, reverse=True)

#         for c in cnts:
#             peri = cv2.arcLength(c, closed=True)
#             approx = cv2.approxPolyDP(c, epsilon=peri*0.02, closed=True)

#             if len(approx) == 4:
#                 docCnt = approx
#                 break

#     contourImage = image.copy()
#     cv2.drawContours(contourImage, [docCnt], -1, (0, 0, 255), 2)
#     # show_images([contourImage], ["Outline"])

#     # Getting the bird's eye view, top-view of the document
#     paper = four_point_transform(image, docCnt.reshape(4, 2))
#     warped = four_point_transform(gray, docCnt.reshape(4, 2))
#     # show_images([paper, warped], ["Paper", "Gray"])

#     # Thresholding the document
#     thresh = cv2.threshold(warped, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
#     # show_images([thresh], ["Thresh"])

#     # Finding contours in threshold image
#     cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
#     cnts = imutils.grab_contours(cnts)
#     print("Total contours found after threshold {}".format(len(cnts)))
#     questionCnts = []
#     allContourImage = paper.copy()
#     cv2.drawContours(allContourImage, cnts, -1, (0, 0, 255), 3)
#     # show_images([allContourImage], ["All contours from threshold image"])

#     # Finding the questions contours
#     for c in cnts:
#         (x, y, w, h) = cv2.boundingRect(c)
#         ar = w / float(h)

#         if w >= 20 and h >= 20 and ar >= 0.9 and ar <= 1.1:
#             questionCnts.append(c)

#     print("Total questions contours found: {}".format(len(questionCnts)))

#     questionsContourImage = paper.copy()
#     cv2.drawContours(questionsContourImage, questionCnts, -1, (0, 0, 255), 3)
#     # show_images([questionsContourImage], ["All questions contours after filtering questions"])

#     # Sorting the contours according to the question
#     questionCnts = contours.sort_contours(questionCnts, method="top-to-bottom")[0]
#     correct = 0
#     questionsContourImage = paper.copy()

#     for (q, i) in enumerate(np.arange(0, len(questionCnts), 5)):
#         cnts = contours.sort_contours(questionCnts[i: i+5])[0]
#         cv2.drawContours(questionsContourImage, cnts, -1, (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)), 2)
#         bubbled = None

#         for (j, c) in enumerate(cnts):
#             mask = np.zeros(thresh.shape, dtype="uint8")
#             cv2.drawContours(mask, [c], -1, 255, -1)

#             mask = cv2.bitwise_and(thresh, thresh, mask=mask)
#             # show_images([mask], ["Mask of question {} for row {}".format(j+1, q+1)])
#             total = cv2.countNonZero(mask)

#             if bubbled is None or total > bubbled[0]:
#                 bubbled = (total, j)

#         color = (0, 0, 255)
#         k = ANSWER_KEY[q]

#         if k == bubbled[1]:
#             color = (0, 255, 0)
#             correct += 1

#         cv2.drawContours(paper, [cnts[k]], -1, color, 3)

#     # show_images([questionsContourImage], ["All questions contours with different colors"])

#     questionNumber = float(len(ANSWER_KEY))
#     score = (correct / questionNumber) * 100
#     print("INFO Score: {:.2f}%".format(score))
#     cv2.putText(paper, "{:.2f}%".format(score), (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)
#     # show_images([image, paper], ["Original", "exam"])
#     return score
    
    
def grader(image, ANSWER_KEY):
    # Edge detection
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    edged = cv2.Canny(blurred, 75, 200)

    # Finding contours in the edge-detected image
    cnts = cv2.findContours(edged, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)

    # Display the total number of contours found
    print("Total contours found after edge detection: {}".format(len(cnts)))

    # Thresholding the grayscale image
    thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]

    # Finding contours in the thresholded image
    cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)

    # Display the total number of contours found
    print("Total contours found after threshold: {}".format(len(cnts)))

    questionCnts = []
    allContourImage = image.copy()
    cv2.drawContours(allContourImage, cnts, -1, (0, 0, 255), 3)

    # Filtering the question contours
    for c in cnts:
        (x, y, w, h) = cv2.boundingRect(c)
        ar = w / float(h)

        if w >= 20 and h >= 20 and ar >= 0.9 and ar <= 1.1:
            questionCnts.append(c)

    # Display the total number of question contours found
    print("Total question contours found: {}".format(len(questionCnts)))

    questionsContourImage = image.copy()
    cv2.drawContours(questionsContourImage, questionCnts, -1, (0, 0, 255), 3)

    # Sorting the contours according to the question
    questionCnts = contours.sort_contours(questionCnts, method="top-to-bottom")[0]
    correct = 0
    questionsContourImage = image.copy()

    for (q, i) in enumerate(np.arange(0, len(questionCnts), 5)):
        cnts = contours.sort_contours(questionCnts[i: i+5])[0]
        cv2.drawContours(questionsContourImage, cnts, -1, (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)), 2)
        bubbled = None

        for (j, c) in enumerate(cnts):
            mask = np.zeros(thresh.shape, dtype="uint8")
            cv2.drawContours(mask, [c], -1, 255, -1)

            mask = cv2.bitwise_and(thresh, thresh, mask=mask)
            total = cv2.countNonZero(mask)

            if bubbled is None or total > bubbled[0]:
                bubbled = (total, j)

        color = (0, 0, 255)
        k = ANSWER_KEY[q]

        if k == bubbled[1]:
            color = (0, 255, 0)
            correct += 1

        cv2.drawContours(image, [cnts[k]], -1, color, 3)

    questionNumber = float(len(ANSWER_KEY))
    score = (correct / questionNumber) * 100
    print("INFO Score: {:.2f}%".format(score))
    cv2.putText(image, "{:.2f}%".format(score), (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)

    return score
    

In [ ]:
# usage

image = cv2.imread( "images/test.png")
ANSWER_KEY = {
    0: 3,
    1: 2,
    2: 1,
}

grader(image,ANSWER_KEY)

In [ ]:
# usage

image = cv2.imread( "images/test_with_frame.png")
ANSWER_KEY = {
    0: 1,
    1: 4,
    2: 0,
    3: 3,
    4: 1
}
grader(image,ANSWER_KEY)

In [ ]:
import cv2
import pytesseract

def calculate_score(answer_sheet_image_path, answer_key_dict):
    # Load the image
    image = cv2.imread(answer_sheet_image_path, 0)  # Load as grayscale

    # Threshold the image to obtain a binary image
    _, binary_image = cv2.threshold(image, 127, 255, cv2.THRESH_BINARY_INV)

    # Find contours of the marked circles
    contours, _ = cv2.findContours(binary_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Sort the contours from left to right
    contours = sorted(contours, key=lambda x: cv2.boundingRect(x)[0])

    # Identify the marked answers
    marked_answers = {}
    for i, contour in enumerate(contours):
        # Compute the centroid of the contour
        M = cv2.moments(contour)
        if M["m00"] != 0:
            cx = int(M["m10"] / M["m00"])
            cy = int(M["m01"] / M["m00"])

            # Determine the question number based on the centroid position
            question_number = int(cx / (image.shape[1] / len(answer_key_dict)))

            # Determine the marked answer based on the centroid position
            marked_answer = int(cy / (image.shape[0] / 5))  # Assuming 5 options per question

            # Store the marked answer for the question
            marked_answers[question_number] = marked_answer

    # Calculate the score by comparing the marked answers with the answer key
    correct_answers = 0
    for question_number, marked_answer in marked_answers.items():
        if question_number in answer_key_dict and marked_answer == answer_key_dict[question_number]:
            correct_answers += 1

    # Calculate the score as the number of correct answers divided by the total number of questions
    total_questions = len(answer_key_dict)
    score = correct_answers / total_questions

    return score


In [ ]:
import cv2
import imutils
import numpy as np
import random

def grader(image, ANSWER_KEY):
    # edge detection
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    edged = cv2.Canny(blurred, 75, 200)

    # find contours in edge detected image
    cnts = cv2.findContours(edged, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    docCnt = None

    # finding the document contour
    if len(cnts) > 0:
        cnts = sorted(cnts, key=cv2.contourArea, reverse=True)

        for c in cnts:
            peri = cv2.arcLength(c, closed=True)
            approx = cv2.approxPolyDP(c, epsilon=peri * 0.02, closed=True)

            if len(approx) == 4:
                docCnt = approx
                break

    # Getting the bird's eye view, top-view of the document
    paper = four_point_transform(image, docCnt.reshape(4, 2))
    warped = four_point_transform(gray, docCnt.reshape(4, 2))

    # Thresholding the document
    thresh = cv2.threshold(warped, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]

    # Finding contours in threshold image
    cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    questionCnts = []

    # Finding the questions contours
    for c in cnts:
        (x, y, w, h) = cv2.boundingRect(c)
        ar = w / float(h)

        if w >= 20 and h >= 20 and ar >= 0.9 and ar <= 1.1:
            questionCnts.append(c)

    # Sorting the contours according to the question
    questionCnts = contours.sort_contours(questionCnts, method="top-to-bottom")[0]
    correct = 0

    # Iterate through groups of 5 question contours
    for (q, i) in enumerate(np.arange(0, len(questionCnts), 5)):
        cnts = contours.sort_contours(questionCnts[i: i + 5])[0]
        bubbled = None

        for (j, c) in enumerate(cnts):
            mask = np.zeros(thresh.shape, dtype="uint8")
            cv2.drawContours(mask, [c], -1, 255, -1)

            mask = cv2.bitwise_and(thresh, thresh, mask=mask)
            total = cv2.countNonZero(mask)

            if bubbled is None or total > bubbled[0]:
                bubbled = (total, j)

        k = ANSWER_KEY[q]

        if k == bubbled[1]:
            correct += 1

    questionNumber = float(len(ANSWER_KEY))
    score = (correct / questionNumber) * 100
    return score


In [ ]:
# usage

image = "images/scan_test.jpg"
ANSWER_KEY = {
    0 :0, 
    1 :0, 
    2 :0, 
    3 :0, 
    4 :0, 
    5 :0, 
    6 :0, 
    7 :0, 
    8 :0, 
}
calculate_score(image,ANSWER_KEY)

In [ ]:
cv2.imread(image)

In [ ]:
answer_sheet_image_path = "images/test_with_frame_with_answers.png"
answer_key_dict = {
    0 :1, 
    1 :1, 
    2 :1, 
    3 :1, 
    4 :1, 
    5 :1, 
    6 :1, 
    7 :1, 
    8 :1, 
}

# Load the image
image = cv2.imread(answer_sheet_image_path, 0)  # Load as grayscale

# Threshold the image to obtain a binary image
_, binary_image = cv2.threshold(image, 127, 255, cv2.THRESH_BINARY_INV)

# Find contours of the marked circles
contours, _ = cv2.findContours(binary_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

contours

# Sort the contours from left to right
contours = sorted(contours, key=lambda x: cv2.boundingRect(x)[0])

# Identify the marked answers
marked_answers = []
for contour in contours:
    # Compute the centroid of the contour
    M = cv2.moments(contour)
    if M["m00"] != 0:
        cx = int(M["m10"] / M["m00"])
        cy = int(M["m01"] / M["m00"])

        # Determine the answer based on the centroid position
        question_number = int(cx / (image.shape[1] / len(answer_key_dict)))
        marked_answer = int(cy / (image.shape[0] / 5))  # Assuming 5 options per question
        marked_answers.append((question_number, marked_answer))
        
# Calculate the score by comparing the marked answers with the answer key
correct_answers = 0
for question_number, marked_answer in marked_answers:
    if question_number in answer_key_dict and marked_answer == answer_key_dict[question_number]:
        correct_answers += 1

# Calculate the score as the number of correct answers divided by the total number of questions
total_questions = len(answer_key_dict)
score = correct_answers / total_questions
